In [1]:
import requests
import urllib.request
import time
import os.path 
from bs4 import BeautifulSoup
import pandas as pd 
from lxml import html
import numpy as np
import re

In [2]:
len_anchors = 494 #only including official episodes. no shorts. keep increasing to 500 until it breaks

size = len_anchors*2000
eps = ["" for x in range(size)]
lines =  ["" for x in range(size)]
names =  ["" for x in range(size)]
dialogue =  ["" for x in range(size)]
actions =  ["" for x in range(size)]

end_of_last_k = 0 
person = ''
split_by_italics = []

In [3]:
for i in range(len_anchors):
    if os.path.exists('./transcripts/spongebob_ep'+str(i)+'.html'):
        html = open('./transcripts/spongebob_ep'+str(i)+'.html','r')
        soup = BeautifulSoup(html, "lxml")
        html.close()
        for script in soup(["script", "style"]):  # kill all script and style elements
            script.extract()    # rip it out
        text = soup.get_text()
        text_by_line = text.split('\n')
        text_no_breaks = [l for l in text_by_line if len(l) > 0]
        #print(text_no_breaks)
        df = pd.DataFrame({'col':text_no_breaks})
        
        remover = df[df['col'].str.contains("h aired on")].index
        remover_end = df[df['col'].str.contains("Start a Discussion")].index
        
        if remover_end.size > 0:  
            remover_end = remover_end[0]-1
        elif remover_end.size == 0:
            remover_end = df[df['col'].str.contains("Retrieved from \"http")].index
            remover_end = remover_end[0]-1     
        dropindex = remover[0]+2
       
        num_lines = len(df)
        line_count = 0
        df = df.truncate(before=dropindex, after=remover_end)
        df.head()
        df.reset_index(inplace=True)
        num_lines = len(df)
        
        for k in range(num_lines):
            eps[k + end_of_last_k] = i
            lines[k + end_of_last_k] = k
            if len(df.iloc[k][1]) > 1:
                line_count = line_count + 1
                split_up = df.iloc[k][1].split(':',maxsplit=1)
                #print(split_up)
                this_lines_actions = []
                words = []
                if len(split_up) > 1:
                    words.append(split_up[1])
                elif len(split_up) == 2:
                    person = split_up[0]
                    #print(person)
                else:
                    continue
                    #print("Error in split!",i,k,text_no_breaks[k])
                names[line_count + end_of_last_k] = person
                more_words = []
                if len(split_up)>=1:
                    split_by_italics = split_up[1].split('[')
                #try taking out if person == french narrator
                if len(split_by_italics) >= 1:
                    for w in range(len(split_by_italics)): #len > 1 indicates actions are present
                        if ']' in split_by_italics[w]:
                            an_action = split_by_italics[w].split(']') 
                            if len(an_action[0]) > 0:
                                thing = []
                                thing.append(an_action[0])
                                this_lines_actions.append(thing)
                            if len(an_action[1]) > 0:
                                thing = []
                                thing.append(an_action[1])
                                more_words.append(thing)
                        else: 
                            if len(split_by_italics[w]) > 1:
                                this_thing = []
                                this_thing.append(split_by_italics[w])
                                more_words.append(this_thing)
                elif len(split_by_italics) == 1:
                        more_words.append(split_by_italics)
                if len(more_words) == 1:
                    dialogue[line_count + end_of_last_k] = more_words[0]
                elif len(more_words) > 1:
                    no_spaces = []
                    for y in range(len(more_words)):
                        if len(more_words[y][0]) > 2:
                            no_spaces.append(more_words[y])
                    dialogue[line_count + end_of_last_k] = no_spaces
                if len(this_lines_actions) == 1:
                    actions[line_count + end_of_last_k] = this_lines_actions[0]
                elif len(this_lines_actions) > 1:
                    actions[line_count + end_of_last_k] = this_lines_actions
                #print("character for line ",k,": ",names[line_count + end_of_last_k])
                #print("words for line ",k,": ",dialogue[line_count + end_of_last_k])
                #print("actions for line ",k,": ",actions[line_count + end_of_last_k]) 
                #print("line ",k,": ",names[line_count + end_of_last_k],": ",dialogue[line_count + end_of_last_k],actions[line_count + end_of_last_k])                      
        end_of_last_k += k
        

In [4]:
print(end_of_last_k)

60858


In [5]:
eps = eps[:end_of_last_k+1]
eps = np.asarray(eps).T 
lines = lines[:end_of_last_k+1]
lines = np.asarray(lines).T
names = names[:end_of_last_k+1]
dialogue = dialogue[:end_of_last_k+1]
actions = actions[:end_of_last_k+1]
names = np.asarray(names).T 
print(len(eps))

60859


In [6]:


data = [[None]*5 for i in range(end_of_last_k+1)]
for i in range(len(data)):
    data[i][0] = eps[i]
    data[i][1] = lines[i]
    data[i][2] = names[i]
    if len(dialogue[i]) > 1:
        # CONCATENATE ELEMENTS OF IT
        s = ', '
        data[i][3] = s.join(["".join(x) for x in dialogue[i]])
    elif len(dialogue[i]) == 1:
        data[i][3] = str(dialogue[i][0])
    elif len(dialogue[i]) == 0:
        data[i][3] = ""
    if len(actions[i]) > 1:
        # CONCATENATE ELEMENTS OF IT
        s = ', '
        data[i][4] = s.join(["".join(x) for x in actions[i]])
    elif len(actions[i]) == 1:
        data[i][4] = str(actions[i][0])
    elif len(actions[i]) == 0:
        data[i][4] = ""
    data[i] = tuple(data[i])

df = pd.DataFrame(data,columns = ['Episode','Line','Character','Dialogue','Action'])
df['Action'] = df['Action'].str.lower() 
df['Dialogue'] = df['Dialogue'].str.lower() 
df.to_pickle("./spongebob_pickle_1.pkl")


In [15]:
df.to_pickle("./spongedf_pickle1.pkl")


,Episode,Line,Character,Dialogue,Action
0,0,0,,,
1,0,1,French Narrator,"ah, the sea... so fascinating. so wonderful. ...","shows from left to right patrick's, squidward'..."
2,0,2,SpongeBob,"today's the big day, gary!",
3,0,3,Gary,meow.,
4,0,4,SpongeBob,"look at me, i'm... , ...naked! , gotta be i...","jumps on the diving board, jumps up, and leave..."
...,...,...,...,...,...
60854,487,95,,,
60855,487,96,Sal,"you'll never get away with this, dr. kelpington!",
60856,487,97,SpongeBob,"mm, could you pass the popcorn?",whispers to the chum monkey
60857,487,98,,,


,Episode,Line,Character,Dialogue,Action
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
60854,False,False,False,False,False
60855,False,False,False,False,False
60856,False,False,False,False,False
60857,False,False,False,False,False


In [9]:
df.Character.unique().shape # 2141 unique characters, but some of these are two characters speaking at once

(2141,)

In [10]:
df.dropna(how='all', inplace=True) #apparently there isn't any fully null rows


,Episode,Line,Character,Dialogue,Action
0,0,0,,,
1,0,1,French Narrator,"ah, the sea... so fascinating. so wonderful. ...","shows from left to right patrick's, squidward'..."
2,0,2,SpongeBob,"today's the big day, gary!",
3,0,3,Gary,meow.,
4,0,4,SpongeBob,"look at me, i'm... , ...naked! , gotta be i...","jumps on the diving board, jumps up, and leave..."
...,...,...,...,...,...
60854,487,95,,,
60855,487,96,Sal,"you'll never get away with this, dr. kelpington!",
60856,487,97,SpongeBob,"mm, could you pass the popcorn?",whispers to the chum monkey
60857,487,98,,,


,Episode,Line,Character,Dialogue,Action
0,0,0,,,
1,0,1,French Narrator,"ah, the sea... so fascinating. so wonderful. ...","shows from left to right patrick's, squidward'..."
2,0,2,SpongeBob,"today's the big day, gary!",
3,0,3,Gary,meow.,
4,0,4,SpongeBob,"look at me, i'm... , ...naked! , gotta be i...","jumps on the diving board, jumps up, and leave..."
...,...,...,...,...,...
60854,487,95,,,
60855,487,96,Sal,"you'll never get away with this, dr. kelpington!",
60856,487,97,SpongeBob,"mm, could you pass the popcorn?",whispers to the chum monkey
60857,487,98,,,
